<a href="https://www.opianalytics.com/"><img src = "https://pythondaymx.github.io/images/opi-analytics.png" width = 300></a>

<h1 align=center><font size = 5>Resolución de caso práctico - Data Scientist</font></h1>

#### Instrucciones:

● Resuelve sólo una pregunta de la sección A, resuelve la pregunta de la sección B y
resuelve el caso de la sección C.

● No se darán puntos adicionales si resuelves más de una pregunta por sección.

● Envía tus respuestas a más tardar 48 horas después de recibir el correo.

● Las respuestas podrán ser enviadas en un PDF, un notebook, o con un link a tu
repositorio de GitHub.

● Debes especificar claramente las respuestas de cada inciso, e indicar el código usado.

<h2 align="center"><b>Sección B<b></h2> 

Descarga la Base de datos histórica de <a href="https://datos.gob.mx/busca/dataset/quien-es-quien-en-los-precios/resource/d89a59ae-a42a-4d71-8c38-dd21d02027ab">Quién es Quién en los Precios</a> de Profeco y resuelve los siguientes incisos. Para el procesamiento de los datos y el análisis exploratorio debes debes usar Spark SQL en el lenguaje de programación de tu elección.

1. Procesamiento de los datos

    a. ¿Cuántos registros hay?
    
    b. ¿Cuántas categorías?
    
    c. ¿Cuántas cadenas comerciales están siendo monitoreadas?
    
    d. ¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de inconsistencia o error en la fuente?
    
    e. ¿Cuáles son los productos más monitoreados en cada entidad?
    
    f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?
    

2. Análisis exploratorio

    a. Genera una canasta de productos básicos que te permita comparar los precios geográfica y temporalmente. Justifica tu elección y procedimiento
    
    b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?
    
    c. ¿Hay algún patrón estacional entre años?
    
    d. ¿Cuál es el estado más caro y en qué mes?
    
    e. ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con estos datos?
    

3. Visualización

    a. Genera un mapa que nos permita identificar la oferta de categorías en la zona metropolitana de León Guanajuato y el nivel de precios en cada una de ellas. Se darán puntos extra si el mapa es interactivo

## Procesamiento de los datos:

Para responder a lo anterior, haremos uso de PySpark y todas las funciones que ésta nos ofrece para el cálculo en grandes volúmenes: 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("Parte B para resolución caso práctico") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [15]:
data = spark.read.csv(
    "all_data.csv", 
    header=True)

In [4]:
data.show(3)

+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.125417|
|            CRAYONE

a) Para determinar la cantidad de datos en la tabla, haremos uso de .count(), así:

In [5]:
data.count()

62530715

Lo que nos indique en la tabla, se tienen **62,530,715** registros.

In [6]:
data.createOrReplaceTempView("Tabla_temporal")

b) Para determinar la cantidad de categorías diferentes en nuestra tabla, hacemos uso de .sql(), ésta nos permite trabajar con la tabla a través de un query; aquí omitiremos las categorías nulas:

In [7]:
spark.sql("SELECT DISTINCT categoria FROM Tabla_temporal where categoria is not null").count()

41

Así, nos dice, que existen **41** categorías diferentes clasificadas de manera oficial; las que figuran como nulas, no entran en ninguna de las **41** categorías.

c) Para éste interrogante, aplicamos la misma lógica, pero en éste caso, para la columna de cadenaComercial:

In [8]:
spark.sql("SELECT DISTINCT cadenaComercial FROM Tabla_temporal where cadenaComercial is not null").count()

705

Así, tenemos que se están monitoreando **705** cadenas comerciales, aquí, los que no registran como cadena comercial, se les asigna el valor nulo.

d) Para determinar la calidad de los datos, podemos hacer un barrido sencillo para los casos donde se registren valores nulos en alguna de las filas en la tabla:

In [9]:
columnas = data.columns

In [16]:
for columna in columnas:
    print(columna)
    print(data.filter(data[columna].isNotNull()).count())
    print("")  

producto
62530715

presentacion
62530715

marca
62530715

categoria
61643377

catalogo
62530487

precio
62530715

fechaRegistro
62530715

cadenaComercial
62529531

giro
62530428

nombreComercial
62530715

direccion
62530715

estado
62515661

municipio
62515661

latitud
62493901

longitud
62493901



Analicemos qué sucede en cada columna con respecto a los valores y en el caso de los valores nulos:

In [30]:
for columna in columnas:
    query_datos =  "SELECT DISTINCT "+columna+" FROM Tabla_temporal order by "+columna+" asc"
    query_vista_5_nulos = "SELECT * FROM Tabla_temporal where "+columna+" is null"
    datos = spark.sql(query_datos)
    datos.show()
    vista_5_nulos = spark.sql(query_vista_5_nulos)
    if vista_5_nulos.count() > 0:
        vista_5_nulos.show(5)

+--------------------+
|            producto|
+--------------------+
|"PREMARIN. CREMA ...|
|             A.S.COR|
|             ABILIFY|
|            ACARBOSA|
|              ACEITE|
|     ACEITE DE OLIVA|
|            ACEITUNA|
|    ACEITUNA. GORDAL|
|ACEITUNA. MANZANILLA|
|ACEITUNA. RELLENA...|
|              ACELGA|
|           ACICLOVIR|
|   ACIDO ALENDRONICO|
|        ACIDO FOLICO|
|ACIDO NALIDIXICO ...|
|           ACLIMAFEL|
|             ACLORAL|
|ACONDICIONADOR / ...|
|ACONDICIONADOR DE...|
|ACONDICIONADOR Y ...|
+--------------------+
only showing top 20 rows

+--------------------+
|        presentacion|
+--------------------+
|# 6017872 O C.B. ...|
|# 6019459  O  C.B...|
|# 6019464 O C.B. ...|
|# 6019660  O C.B....|
|# 6020580 O C.B. ...|
|#703377 O 75484 O...|
|#703378 O 75491 O...|
|#703470  O  66079...|
|#706856 Ó MOD. 75...|
|''F''  CAJA CON 1...|
|     00009. MONOPOLY|
|00322. CUBO DE FO...|
|0140808. PATIN BA...|
|0140819. PATIN/SC...|
|0160133. TRICICLO...|
|0160808

+--------------+--------------------+---------+------------+------------+------+--------------------+---------------+--------------------+--------------------+--------------------+------+--------------------+-------+--------+
|      producto|        presentacion|    marca|   categoria|    catalogo|precio|       fechaRegistro|cadenaComercial|                giro|     nombreComercial|           direccion|estado|           municipio|latitud|longitud|
+--------------+--------------------+---------+------------+------------+------+--------------------+---------------+--------------------+--------------------+--------------------+------+--------------------+-------+--------+
|  DOLOCARTIGEN|CAJA CON 20 CAPSU...|      S/M|MEDICAMENTOS|MEDICAMENTOS| 586.4|2013-05-10 00:00:...|           null|FARMACIA BOTICA Y...|FARMACIA NUEVA IM...|AV. SAN FERNANDO ...|COLIMA|COLIMA           ...|     NA|      NA|
|CELESTAMINE-NS|CAJA CON FRASCO 6...|      S/M|MEDICAMENTOS|MEDICAMENTOS|279.28|2013-05-10 00:00

+--------------------+
|           municipio|
+--------------------+
|                null|
|          CP. 27280"|
|ACAPULCO DE JUARE...|
|  ACAPULCO DE JUÁREZ|
|         AGUA PRIETA|
|AGUA PRIETA      ...|
|      AGUASCALIENTES|
|AGUASCALIENTES   ...|
|ALVARO OBREGON   ...|
|             APIZACO|
|APIZACO          ...|
|             APODACA|
|APODACA          ...|
|ATIZAPAN         ...|
|ATIZAPAN DE ZARAG...|
|            ATIZAPÁN|
|ATIZAPÁN DE ZARAGOZA|
|        AZCAPOTZALCO|
|AZCAPOTZALCO     ...|
|BENITO JUAREZ    ...|
+--------------------+
only showing top 20 rows

+--------------------+--------------------+-------------------+---------------+--------+------+--------------------+------------------+--------------------+------------------+--------------------+------+---------+-------+--------+
|            producto|        presentacion|              marca|      categoria|catalogo|precio|       fechaRegistro|   cadenaComercial|                giro|   nombreComercial|           direc

Con éste loop pudimos identificar que en los datos, existían numerosos valores nulos en varias columnas, como por ejemplo, la columna de **categoria**, seguida de las columnas de **latitud** y **longitud**.

Además, pudimos observar que en cada columna, existian valores, que aunque no son nulos, son inconsistentes, ejemplo; para la columna *producto* existe un producto que en esa columna solo es nombrado **producto**, al igual que en la columna de *catalogo*; otro detalle por ejemplo, es que en la columna *estado* existen registros de una entidad federativa llamada **ESQ. SUR 125"**, que no registra como ninguna de las entidades federativas del gobierno mexicano.

Además, en caso de hacerse la respectiva eliminación de todos esos registros nulos (por columna), nos quedará en total:

In [59]:
for columna in columnas:
    data = data.filter(data[columna].isNotNull())
data.count()

61593576

Con esto, nos quedamos con cerca del **98,5%** de los datos originales,de los cuales, fueron suprimidos por arrojar valores nulos en alguna de sus columnas.

   e) Para observar cuales son los productos más monitoreados basta con hacer una consulta con .sql :

In [ ]:
entidades = spark.sql("SELECT distinct(estado) as entidad from Tabla_temporal").select('entidad').rdd.flatMap(lambda x: x).collect()

In [57]:
entidades = list(filter(None, entidades))

Aquí podemos ver los 3 productos más monitoreados por cada entidad federativa:

In [58]:
for entidad in entidades:
    query_por_entidad = "SELECT producto, count(producto) as cantidad FROM Tabla_temporal where producto is not null and estado = '"+entidad+"' and estado is not null GROUP BY producto ORDER BY cantidad desc"
    data_entidad = spark.sql(query_por_entidad)
    print (entidad)
    data_entidad.show(3)
   

QUINTANA ROO
+---------+--------+
| producto|cantidad|
+---------+--------+
|      FUD|   34846|
| REFRESCO|   34367|
|LAVADORAS|   32347|
+---------+--------+
only showing top 3 rows

NUEVO LEÓN
+--------------------+--------+
|            producto|cantidad|
+--------------------+--------+
|   DETERGENTE P/ROPA|   50307|
|            REFRESCO|   49592|
|LECHE ULTRAPASTEU...|   43803|
+--------------------+--------+
only showing top 3 rows

SINALOA
+-----------------+--------+
|         producto|cantidad|
+-----------------+--------+
|         REFRESCO|   33115|
|DETERGENTE P/ROPA|   27177|
|          SHAMPOO|   22435|
+-----------------+--------+
only showing top 3 rows

TABASCO
+-----------------+--------+
|         producto|cantidad|
+-----------------+--------+
|         REFRESCO|   28754|
|DETERGENTE P/ROPA|   26431|
|        LAVADORAS|   26361|
+-----------------+--------+
only showing top 3 rows

BAJA CALIFORNIA
+-----------------+--------+
|         producto|cantidad|
+--------

f) Para ésta pregunta, veremos:

In [67]:
query_cadena_mayor_variedad = "SELECT cadenaComercial,count(DISTINCT(producto)) AS productos_distintos from Tabla_temporal GROUP BY cadenaComercial ORDER BY productos_distintos desc"
spark.sql(query_cadena_mayor_variedad).show(10)

+--------------------+-------------------+
|     cadenaComercial|productos_distintos|
+--------------------+-------------------+
|             SORIANA|               1059|
|            WAL-MART|               1051|
|MEGA COMERCIAL ME...|               1049|
|  COMERCIAL MEXICANA|               1036|
|            CHEDRAUI|               1026|
|     MERCADO SORIANA|               1024|
|      BODEGA AURRERA|               1012|
|HIPERMERCADO SORIANA|               1006|
|              H.E.B.|               1001|
|        SORIANA PLUS|                999|
+--------------------+-------------------+
only showing top 10 rows



Con esto vemos que la cadena comercial con mayor variedad de productos monitoreados es **Soriana** con **1059** artículos distintos registrados.

## Análisis Exploratorio

a) Para poder determinar una canasta de productos básicos, debemos sustraer los productos que estén en ese catálogo, y que además, existan registro de ellos en cada una de las entidades federativas.

In [72]:
data_tiempo_geo = spark.sql("SELECT producto,presentacion, avg(precio), fechaRegistro, estado, municipio from Tabla_temporal where catalogo='BASICOS' AND precio is not null GROUP BY producto, fechaRegistro, estado, municipio, presentacion")
data_tiempo_geo.show(10)

+--------------------+--------------------+---------------------------+--------------------+----------------+--------------------+
|            producto|        presentacion|avg(CAST(precio AS DOUBLE))|       fechaRegistro|          estado|           municipio|
+--------------------+--------------------+---------------------------+--------------------+----------------+--------------------+
|   GELATINA EN POLVO|CAJA 84 GR. AGUA....|                        6.7|2011-01-10 00:00:...|          MÉXICO|ATIZAPAN         ...|
|       SALSA PICANTE|BOTELLA 370 ML. E...|                        5.0|2011-01-10 00:00:...|          MÉXICO|ATIZAPAN         ...|
|      CHILES EN LATA|LATA 215 GR. JALA...|          6.066666666666666|2011-01-10 00:00:...|DISTRITO FEDERAL|BENITO JUAREZ    ...|
|    JABON DE TOCADOR|CAJA CON BARRA 10...|                       12.1|2011-01-10 00:00:...|DISTRITO FEDERAL|BENITO JUAREZ    ...|
|      CHILES EN LATA|LATA 220 GR. JALA...|          6.013333333333333|2011-01-10 0

Con esto, tenemos los productos que se registran en todas las ciudades y sus precios promedio por fecha, para proceder, debemos determinar cuales son esos productos que están en todas las ciudades y luego, de los que satisfacen esa condición, serán los productos de la canasta básica mexicana.

Posteriormente 
Procedemos de esa forma para poder deshacernos de aquellos productos que no se tienen registro, salvo en ciertas zonas específicas. Para hablar de una canasta familiar mexicana, se debe realizar como lo anterior, algo que se consuma en todos lados y que se venden en todos los lugares de las entidades federativas.

Además, como queremos analizar producto, lugar y tiempo específicos, debemos extraer la ubicación al menos por estado y municipio, esto para establecer un precio promedio en una misma ciudad.

b) Para esto, tomamos los precios promedios registrados a la fecha más reciente (lo ideal seria considerar todo un mes completo, no una fecha específica) en los registros de cada ciudad, sumamos estos por municipio y éste será el que nos determinará el costo de una canasta de productos básicos; así, se toma el municipio con el indicidor más alto y con el indicador más bajo. 

c) Agrupamos por meses para cada año el costo total promedio de la canasta de productos básico, desarrollamos una gráfica y observamos por algún patrón entre las series de tiempo.

d) Para esto, consideremos el comportamiento del indicador para el último mes; entonces tomamos el último mes registrado, agrupamos por estado y tomamos los precios promedios agrupados por estado, sumamos los valores de cada elemento en la canasta de productos básicos; así, se toma el estado con el indicador más alto.

e) La ausencia de datos en periodos de tiempo, ya que pueden existir registros donde la frecuencia de muestreo es menor que en otras; así, si se deseara comparar precios a día, surgirán errores.

## Visualización

Ésta parte se puede desarrollar filtrando la información para el municipio de León, Guanajuato; agrupamos por categoría, determinamos el precio promedio por categoría en cada comercio, y señalamos ésta información en el comercio(nombreComercial) y su ubicación en un mapa.

Lo ideal, sería implementar una dropdown para la lista de categorías y sean éstas las que se muestren en el popup que hace referencia al comercio.